# ⚡ SQL OPTIMIZATION

---

## 📋 **DAY 5 - LESSON 4: SQL OPTIMIZATION**

### **🎯 MỤC TIÊU:**

1. **Query Execution Plans** - Understand EXPLAIN
2. **Cost-Based Optimization** - Statistics and CBO
3. **Predicate Pushdown** - Filter early
4. **Join Optimization** - Broadcast, Sort-Merge, Shuffle Hash
5. **Partition Pruning** - Skip unnecessary partitions
6. **Query Hints** - Control execution
7. **Common Anti-Patterns** - What to avoid
8. **Performance Tuning** - Best practices

---

## 💡 **SQL OPTIMIZATION:**

- Understand how Spark executes queries
- Use EXPLAIN to analyze plans
- Optimize joins and filters
- Avoid common mistakes
- Tune for performance

---

## 🔧 **SETUP**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, when, desc, asc, broadcast
from pyspark.sql.types import *
import time
import random
from datetime import datetime, timedelta

spark = SparkSession.builder \
    .appName("SQLOptimization") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "1g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.autoBroadcastJoinThreshold", "10485760") \
    .config("spark.sql.cbo.enabled", "true") \
    .config("spark.sql.statistics.histogram.enabled", "true") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark Session Created")
print(f"Spark Version: {spark.version}")
print(f"Adaptive Execution: {spark.conf.get('spark.sql.adaptive.enabled')}")
print(f"CBO Enabled: {spark.conf.get('spark.sql.cbo.enabled')}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 16:49:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/11 16:49:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✅ Spark Session Created
Spark Version: 3.5.1
Adaptive Execution: true
CBO Enabled: true


---

## 📊 **1. TẠO DATA MẪU**

In [ ]:
print("="*80)
print("📊 1. GENERATING SAMPLE DATA")
print("="*80)

# Large employees dataset
print("\n🔹 Generating 10,000 employees...")

departments = ["Engineering", "Sales", "Marketing", "HR", "Finance"]
countries = ["USA", "UK", "Germany", "France", "Canada"]
cities = {
    "USA": ["New York", "San Francisco", "Seattle"],
    "UK": ["London", "Manchester"],
    "Germany": ["Berlin", "Munich"],
    "France": ["Paris", "Lyon"],
    "Canada": ["Toronto", "Vancouver"]
}

employees_data = []
for i in range(1, 10001):
    country = random.choice(countries)
    city = random.choice(cities[country])
    dept = random.choice(departments)
    
    base_salary = {
        "Engineering": 80000,
        "Sales": 70000,
        "Marketing": 65000,
        "HR": 60000,
        "Finance": 75000
    }[dept]
    
    employees_data.append((
        f"EMP{i:05d}",
        f"Employee {i}",
        random.randint(22, 60),
        dept,
        country,
        city,
        base_salary + random.randint(-10000, 30000),
        random.choice(["Active", "Active", "Active", "Inactive"]),
        (datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1460))).strftime("%Y-%m-%d")
    ))

employees = spark.createDataFrame(employees_data,
    ["employee_id", "name", "age", "department", "country", "city", "salary", "status", "hire_date"])

print(f"✅ Generated {employees.count():,} employees")

# Small departments lookup table
print("\n🔹 Generating departments lookup...")

departments_data = [
    ("Engineering", "Tech", "John Doe"),
    ("Sales", "Business", "Jane Smith"),
    ("Marketing", "Business", "Bob Johnson"),
    ("HR", "Support", "Alice Brown"),
    ("Finance", "Support", "Charlie Wilson")
]

departments_lookup = spark.createDataFrame(departments_data,
    ["department", "division", "manager"])

print(f"✅ Generated {departments_lookup.count()} departments")

# Create temporary views
employees.createOrReplaceTempView("employees")
departments_lookup.createOrReplaceTempView("departments")

print("\n✅ Created temporary views")

📊 1. GENERATING SAMPLE DATA

🔹 Generating 10,000 employees...


26/01/11 16:50:05 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/11 16:50:20 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/11 16:50:35 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/11 16:50:50 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/11 16:51:05 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
26/01/11 16:51:20 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

---

## 📖 **2. QUERY EXECUTION PLANS**

### **Understanding EXPLAIN:**
- **Parsed Logical Plan**: SQL → Logical plan
- **Analyzed Logical Plan**: Resolve columns, tables
- **Optimized Logical Plan**: Apply optimizations
- **Physical Plan**: How to execute

In [ ]:
print("="*80)
print("📖 2. QUERY EXECUTION PLANS")
print("="*80)

# Simple query
query = """
    SELECT department, COUNT(*) as count, AVG(salary) as avg_salary
    FROM employees
    WHERE status = 'Active'
    GROUP BY department
    ORDER BY avg_salary DESC
"""

result = spark.sql(query)

print("\n📊 A. SIMPLE EXPLAIN")
print("-" * 80)
result.explain()

print("\n📊 B. EXTENDED EXPLAIN (All stages)")
print("-" * 80)
result.explain(extended=True)

print("\n📊 C. FORMATTED EXPLAIN (Easy to read)")
print("-" * 80)
result.explain(mode="formatted")

print("\n📊 D. COST EXPLAIN (With statistics)")
print("-" * 80)
result.explain(mode="cost")

In [ ]:
# Explain modes comparison
print("\n📊 EXPLAIN MODES COMPARISON")
print("-" * 80)

print("""
💡 EXPLAIN MODES:

1. explain() - Simple
   - Shows physical plan only
   - Easy to read
   - Good for quick check

2. explain(extended=True) - Extended
   - Shows all 4 stages:
     * Parsed Logical Plan
     * Analyzed Logical Plan
     * Optimized Logical Plan
     * Physical Plan
   - Good for debugging

3. explain(mode="formatted") - Formatted
   - Tree structure
   - Shows node IDs
   - Easy to understand flow

4. explain(mode="cost") - Cost
   - Shows statistics
   - Row counts, data sizes
   - Good for optimization

5. explain(mode="codegen") - Code Generation
   - Shows generated Java code
   - Advanced debugging
""")

---

## 📊 **3. COST-BASED OPTIMIZATION (CBO)**

### **What is CBO?**
- Uses statistics to choose best execution plan
- Estimates cost of different strategies
- Chooses cheapest plan

### **Statistics:**
- Table statistics: row count, size
- Column statistics: min, max, distinct count, nulls

In [ ]:
print("="*80)
print("📊 3. COST-BASED OPTIMIZATION")
print("="*80)

# A. Compute statistics
print("\n📊 A. COMPUTE STATISTICS")
print("-" * 80)

# Table statistics
print("\n1. Computing table statistics...")
spark.sql("ANALYZE TABLE employees COMPUTE STATISTICS")
print("✅ Table statistics computed")

# Column statistics
print("\n2. Computing column statistics...")
spark.sql("""
    ANALYZE TABLE employees 
    COMPUTE STATISTICS FOR COLUMNS 
    department, country, salary, age, status
""")
print("✅ Column statistics computed")

In [ ]:
# B. View statistics
print("\n📊 B. VIEW STATISTICS")
print("-" * 80)

# Table stats
print("\n1. Table statistics:")
spark.sql("DESCRIBE EXTENDED employees").filter(
    col("col_name").contains("Statistics")
).show(truncate=False)

# Column stats
print("\n2. Column statistics (salary):")
spark.sql("DESCRIBE EXTENDED employees salary").show(truncate=False)

In [ ]:
# C. Compare with and without statistics
print("\n📊 C. COMPARE PLANS (With vs Without Statistics)")
print("-" * 80)

query = """
    SELECT e.department, d.division, COUNT(*) as count
    FROM employees e
    JOIN departments d ON e.department = d.department
    WHERE e.salary > 80000
    GROUP BY e.department, d.division
"""

print("\nWith statistics (CBO enabled):")
spark.sql(query).explain(mode="cost")

print("""
💡 CBO Benefits:
   - Better join strategy selection
   - Accurate cardinality estimation
   - Optimal filter ordering
   - Better resource allocation
""")

---

## 🔍 **4. PREDICATE PUSHDOWN**

### **What is Predicate Pushdown?**
- Push filters as early as possible
- Reduce data processed
- Happens automatically in Spark

### **Benefits:**
- Less data to read
- Less data to shuffle
- Faster queries

In [ ]:
print("="*80)
print("🔍 4. PREDICATE PUSHDOWN")
print("="*80)

# Bad: Filter after aggregation
print("\n❌ BAD: Filter after aggregation")
print("-" * 80)

bad_query = spark.sql("""
    SELECT department, COUNT(*) as count
    FROM employees
    GROUP BY department
""").filter(col("count") > 1000)

print("Plan:")
bad_query.explain()

start = time.time()
bad_result = bad_query.collect()
bad_time = time.time() - start
print(f"\nTime: {bad_time:.3f}s")

In [ ]:
# Good: Filter before aggregation
print("\n✅ GOOD: Filter before aggregation")
print("-" * 80)

good_query = spark.sql("""
    SELECT department, COUNT(*) as count
    FROM employees
    WHERE status = 'Active'
    GROUP BY department
    HAVING COUNT(*) > 1000
""")

print("Plan:")
good_query.explain()

start = time.time()
good_result = good_query.collect()
good_time = time.time() - start
print(f"\nTime: {good_time:.3f}s")

print(f"\n📊 Speedup: {bad_time/good_time:.2f}x faster")

In [ ]:
# Predicate pushdown examples
print("\n📊 PREDICATE PUSHDOWN EXAMPLES")
print("-" * 80)

print("""
💡 PREDICATE PUSHDOWN:

1. Column Pruning
   ❌ SELECT * FROM table
   ✅ SELECT col1, col2 FROM table

2. Filter Pushdown
   ❌ SELECT * FROM table → Filter in Python
   ✅ SELECT * FROM table WHERE condition

3. Projection Pushdown
   ❌ Read all columns → Select needed
   ✅ Read only needed columns

4. Partition Pruning
   ❌ Scan all partitions
   ✅ Scan only matching partitions

Spark does this automatically!
But you can help by:
   - Filtering early
   - Selecting only needed columns
   - Using partition columns in filters
""")

---

## 🔗 **5. JOIN OPTIMIZATION**

### **Join Strategies:**
1. **Broadcast Hash Join** - Small table broadcast to all nodes
2. **Sort-Merge Join** - Sort both sides, then merge
3. **Shuffle Hash Join** - Hash partition both sides

### **When to use:**
- Broadcast: One table < 10MB
- Sort-Merge: Large tables, sorted data
- Shuffle Hash: Large tables, not sorted

In [ ]:
print("="*80)
print("🔗 5. JOIN OPTIMIZATION")
print("="*80)

# A. Broadcast Join (Small table)
print("\n📊 A. BROADCAST JOIN")
print("-" * 80)

# Without hint
print("\n1. Without broadcast hint:")
query_no_hint = spark.sql("""
    SELECT e.*, d.division, d.manager
    FROM employees e
    JOIN departments d ON e.department = d.department
    WHERE e.salary > 80000
""")

query_no_hint.explain()

start = time.time()
count_no_hint = query_no_hint.count()
time_no_hint = time.time() - start
print(f"\nTime: {time_no_hint:.3f}s")

In [ ]:
# With broadcast hint
print("\n2. With broadcast hint:")
query_with_hint = spark.sql("""
    SELECT /*+ BROADCAST(d) */ e.*, d.division, d.manager
    FROM employees e
    JOIN departments d ON e.department = d.department
    WHERE e.salary > 80000
""")

query_with_hint.explain()

start = time.time()
count_with_hint = query_with_hint.count()
time_with_hint = time.time() - start
print(f"\nTime: {time_with_hint:.3f}s")

if time_no_hint > time_with_hint:
    print(f"\n✅ Broadcast join is {time_no_hint/time_with_hint:.2f}x faster!")
else:
    print(f"\n💡 Similar performance (Spark may auto-broadcast)")

In [ ]:
# B. DataFrame API broadcast
print("\n📊 B. BROADCAST IN DATAFRAME API")
print("-" * 80)

# Using broadcast function
result_broadcast = employees.join(
    broadcast(departments_lookup),
    "department"
).filter(col("salary") > 80000)

print("\nPlan with broadcast():")
result_broadcast.explain()

print("""
💡 BROADCAST JOIN:
   - Best for small tables (< 10MB)
   - No shuffle needed
   - Much faster
   - Use /*+ BROADCAST(table) */ in SQL
   - Use broadcast(df) in DataFrame API
""")

In [ ]:
# C. Join strategies comparison
print("\n📊 C. JOIN STRATEGIES COMPARISON")
print("-" * 80)

print("""
💡 JOIN STRATEGIES:

1. BROADCAST HASH JOIN
   When: One table < 10MB (default threshold)
   How: Broadcast small table to all executors
   Pros: No shuffle, very fast
   Cons: Limited by broadcast size
   
2. SORT-MERGE JOIN
   When: Both tables large, sorted
   How: Sort both sides, merge sorted data
   Pros: Good for large tables
   Cons: Requires sort (expensive)
   
3. SHUFFLE HASH JOIN
   When: Both tables large, not sorted
   How: Hash partition both sides
   Pros: No sort needed
   Cons: Requires shuffle

Spark chooses automatically based on:
   - Table sizes
   - Statistics
   - Sort order
   - Available memory
""")

---

## 🎯 **6. QUERY HINTS**

### **Available Hints:**
- **BROADCAST**: Force broadcast join
- **MERGE**: Force sort-merge join
- **SHUFFLE_HASH**: Force shuffle hash join
- **SHUFFLE_REPLICATE_NL**: Force shuffle replicate nested loop join
- **COALESCE**: Reduce number of partitions
- **REPARTITION**: Increase number of partitions

In [ ]:
print("="*80)
print("🎯 6. QUERY HINTS")
print("="*80)

# A. Broadcast hint
print("\n📊 A. BROADCAST HINT")
print("-" * 80)

spark.sql("""
    SELECT /*+ BROADCAST(d) */ *
    FROM employees e
    JOIN departments d ON e.department = d.department
""").explain()

# B. Merge hint
print("\n📊 B. MERGE HINT (Sort-Merge Join)")
print("-" * 80)

spark.sql("""
    SELECT /*+ MERGE(e, d) */ *
    FROM employees e
    JOIN departments d ON e.department = d.department
""").explain()

# C. Shuffle hash hint
print("\n📊 C. SHUFFLE_HASH HINT")
print("-" * 80)

spark.sql("""
    SELECT /*+ SHUFFLE_HASH(e, d) */ *
    FROM employees e
    JOIN departments d ON e.department = d.department
""").explain()

In [ ]:
# D. Coalesce and Repartition hints
print("\n📊 D. COALESCE AND REPARTITION HINTS")
print("-" * 80)

# Coalesce hint
print("\n1. COALESCE hint (reduce partitions):")
spark.sql("""
    SELECT /*+ COALESCE(2) */ department, COUNT(*) as count
    FROM employees
    GROUP BY department
""").explain()

# Repartition hint
print("\n2. REPARTITION hint (increase partitions):")
spark.sql("""
    SELECT /*+ REPARTITION(10) */ department, COUNT(*) as count
    FROM employees
    GROUP BY department
""").explain()

# Repartition by column
print("\n3. REPARTITION by column:")
spark.sql("""
    SELECT /*+ REPARTITION(department) */ *
    FROM employees
""").explain()

In [ ]:
# E. Multiple hints
print("\n📊 E. MULTIPLE HINTS")
print("-" * 80)

spark.sql("""
    SELECT /*+ BROADCAST(d), COALESCE(2) */ 
        e.department, d.division, COUNT(*) as count
    FROM employees e
    JOIN departments d ON e.department = d.department
    GROUP BY e.department, d.division
""").explain()

print("""
💡 QUERY HINTS:
   - Use hints to control execution
   - Spark usually chooses well automatically
   - Use hints when you know better
   - Test with and without hints
   - Hints are suggestions, not commands
""")

---

## ❌ **7. COMMON ANTI-PATTERNS**

### **What to Avoid:**
1. SELECT *
2. No filters
3. Cross joins
4. UDFs instead of built-in functions
5. Collecting large datasets
6. Not using partitioning
7. Skewed data
8. Too many small files

In [ ]:
print("="*80)
print("❌ 7. COMMON ANTI-PATTERNS")
print("="*80)

print("""
💡 COMMON ANTI-PATTERNS:

1. SELECT * (Column Pruning)
   ❌ SELECT * FROM large_table
   ✅ SELECT col1, col2 FROM large_table
   Why: Reads unnecessary data

2. No Filters (Predicate Pushdown)
   ❌ SELECT * FROM table → Filter in Python
   ✅ SELECT * FROM table WHERE condition
   Why: Processes all data

3. Cross Joins
   ❌ SELECT * FROM t1 CROSS JOIN t2
   ✅ SELECT * FROM t1 JOIN t2 ON t1.id = t2.id
   Why: Cartesian product is huge

4. UDFs Instead of Built-in Functions
   ❌ udf(lambda x: x.upper())
   ✅ F.upper(col("name"))
   Why: UDFs are slow (Python → JVM)

5. Collecting Large Datasets
   ❌ df.collect()  # 1TB data
   ✅ df.limit(100).collect()
   Why: OOM error

6. Not Using Partitioning
   ❌ df.write.parquet("path")
   ✅ df.write.partitionBy("date").parquet("path")
   Why: Slow queries

7. Skewed Data
   ❌ One partition has 90% of data
   ✅ Use salting or AQE
   Why: One task is slow

8. Too Many Small Files
   ❌ 10,000 files of 1KB each
   ✅ 100 files of 100KB each
   Why: Metadata overhead
""")

In [ ]:
# Demo: SELECT * vs SELECT columns
print("\n📊 DEMO: SELECT * vs SELECT COLUMNS")
print("-" * 80)

# Bad: SELECT *
print("\n❌ Bad: SELECT *")
start = time.time()
result_bad = spark.sql("SELECT * FROM employees WHERE salary > 80000")
count_bad = result_bad.count()
time_bad = time.time() - start
print(f"Time: {time_bad:.3f}s")

# Good: SELECT specific columns
print("\n✅ Good: SELECT specific columns")
start = time.time()
result_good = spark.sql("""
    SELECT employee_id, name, department, salary 
    FROM employees 
    WHERE salary > 80000
""")
count_good = result_good.count()
time_good = time.time() - start
print(f"Time: {time_good:.3f}s")

if time_bad > time_good:
    print(f"\n✅ Selecting columns is {time_bad/time_good:.2f}x faster!")

In [ ]:
# Demo: Built-in function vs UDF
print("\n📊 DEMO: BUILT-IN FUNCTION vs UDF")
print("-" * 80)

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Bad: UDF
print("\n❌ Bad: UDF")
upper_udf = udf(lambda x: x.upper() if x else None, StringType())

start = time.time()
result_udf = employees.withColumn("name_upper", upper_udf(col("name")))
count_udf = result_udf.count()
time_udf = time.time() - start
print(f"Time: {time_udf:.3f}s")

# Good: Built-in function
print("\n✅ Good: Built-in function")
start = time.time()
result_builtin = employees.withColumn("name_upper", F.upper(col("name")))
count_builtin = result_builtin.count()
time_builtin = time.time() - start
print(f"Time: {time_builtin:.3f}s")

if time_udf > time_builtin:
    print(f"\n✅ Built-in function is {time_udf/time_builtin:.2f}x faster!")

---

## ⚡ **8. PERFORMANCE TUNING BEST PRACTICES**

In [ ]:
print("="*80)
print("⚡ 8. PERFORMANCE TUNING BEST PRACTICES")
print("="*80)

print("""
💡 PERFORMANCE TUNING CHECKLIST:

1. DATA READING
   ✅ Use columnar formats (Parquet, ORC)
   ✅ Partition data by frequently filtered columns
   ✅ Use predicate pushdown (filter early)
   ✅ Select only needed columns
   ✅ Use compression (snappy, gzip)

2. JOINS
   ✅ Broadcast small tables (< 10MB)
   ✅ Use appropriate join strategy
   ✅ Filter before joining
   ✅ Join on partition keys when possible
   ✅ Avoid cross joins

3. AGGREGATIONS
   ✅ Filter before aggregating
   ✅ Use partial aggregations
   ✅ Consider approximate aggregations (approx_count_distinct)
   ✅ Use appropriate number of partitions

4. SHUFFLES
   ✅ Minimize shuffles
   ✅ Use appropriate partition size (128MB-256MB)
   ✅ Coalesce after filtering
   ✅ Repartition before expensive operations

5. CACHING
   ✅ Cache frequently accessed data
   ✅ Use appropriate storage level
   ✅ Unpersist when done
   ✅ Monitor cache memory usage

6. STATISTICS
   ✅ Compute table statistics
   ✅ Compute column statistics
   ✅ Enable CBO
   ✅ Update statistics regularly

7. CONFIGURATION
   ✅ Enable Adaptive Query Execution (AQE)
   ✅ Set appropriate memory
   ✅ Configure parallelism
   ✅ Tune broadcast threshold

8. MONITORING
   ✅ Use Spark UI
   ✅ Check execution plans
   ✅ Monitor stage times
   ✅ Identify bottlenecks
   ✅ Profile queries
""")

In [ ]:
# Configuration recommendations
print("\n📊 RECOMMENDED SPARK CONFIGURATIONS")
print("-" * 80)

print("""
💡 RECOMMENDED CONFIGURATIONS:

# Adaptive Query Execution
spark.sql.adaptive.enabled = true
spark.sql.adaptive.coalescePartitions.enabled = true
spark.sql.adaptive.skewJoin.enabled = true

# Cost-Based Optimization
spark.sql.cbo.enabled = true
spark.sql.statistics.histogram.enabled = true

# Broadcast
spark.sql.autoBroadcastJoinThreshold = 10485760  # 10MB

# Shuffle
spark.sql.shuffle.partitions = 200  # Adjust based on data size
spark.sql.files.maxPartitionBytes = 134217728  # 128MB

# Memory
spark.executor.memory = 4g
spark.driver.memory = 2g
spark.memory.fraction = 0.6

# Compression
spark.sql.parquet.compression.codec = snappy

# Dynamic Allocation
spark.dynamicAllocation.enabled = true
spark.dynamicAllocation.minExecutors = 1
spark.dynamicAllocation.maxExecutors = 10
""")

---

## 🎓 **KEY TAKEAWAYS**

### **✅ What You Learned:**

1. **Query Execution Plans**
   - Use EXPLAIN to understand queries
   - 4 stages: Parsed, Analyzed, Optimized, Physical
   - Different explain modes

2. **Cost-Based Optimization**
   - Compute statistics for better plans
   - Table and column statistics
   - Enable CBO

3. **Predicate Pushdown**
   - Filter early
   - Select only needed columns
   - Spark does this automatically

4. **Join Optimization**
   - Broadcast small tables
   - Choose appropriate join strategy
   - Use hints when needed

5. **Query Hints**
   - BROADCAST, MERGE, SHUFFLE_HASH
   - COALESCE, REPARTITION
   - Use sparingly

6. **Anti-Patterns**
   - Avoid SELECT *
   - Filter early
   - Use built-in functions
   - Avoid cross joins

7. **Performance Tuning**
   - Enable AQE and CBO
   - Compute statistics
   - Monitor with Spark UI
   - Tune configurations

### **📊 Quick Reference:**

```python
# Explain query
df.explain()
df.explain(extended=True)
df.explain(mode="formatted")
df.explain(mode="cost")

# Compute statistics
spark.sql("ANALYZE TABLE table_name COMPUTE STATISTICS")
spark.sql("ANALYZE TABLE table_name COMPUTE STATISTICS FOR COLUMNS col1, col2")

# Broadcast join
df1.join(broadcast(df2), "key")
spark.sql("SELECT /*+ BROADCAST(t2) */ * FROM t1 JOIN t2 ON t1.id = t2.id")

# Query hints
/*+ BROADCAST(table) */
/*+ MERGE(t1, t2) */
/*+ SHUFFLE_HASH(t1, t2) */
/*+ COALESCE(n) */
/*+ REPARTITION(n) */
```

### **🚀 Congratulations!**

You've completed **DAY 5: SPARK SQL**!

---

In [ ]:
# Cleanup
spark.catalog.clearCache()
spark.stop()

print("✅ Spark session stopped")
print("\n🎉 DAY 5 - LESSON 4 COMPLETED!")
print("\n💡 Remember:")
print("   - Always use EXPLAIN to understand queries")
print("   - Compute statistics for better optimization")
print("   - Broadcast small tables")
print("   - Filter early, select only needed columns")
print("   - Avoid anti-patterns")
print("   - Monitor with Spark UI")
print("\n🔥 Quote: 'Premature optimization is the root of all evil, but knowing how to optimize is essential!' ⚡")
print("\n🎊 CONGRATULATIONS! You've completed DAY 5: SPARK SQL! 🎊")